In [46]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # Library for web scraping

print('Libraries imported.')

Libraries imported.


In [47]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv
#!pip install  html5lib
print('BeautifulSoup  & csv imported.')

BeautifulSoup  & csv imported.


In [48]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
print('SSL certificate errors ignored.')

SSL certificate errors ignored.


In [49]:
source = requests.get('https://www.mylondon.news/news/health/london-coronavirus-map-exact-number-18148424').text
    
soup = BeautifulSoup(source, 'lxml')

print(soup.prettify())

<!DOCTYPE html>
<html>
 <!--
page generated at: Sat Apr 25 17:00:16 BST 2020
page generated by escenic.server/hostname: reg-pres215.tm-aws.com/reg-pres215.tm-aws.com
page generated in section: 51153
-->
 <head>
  <link href="https://s2-prod.mylondon.news" rel="dns-prefetch"/>
  <link href="https://s2-prod.mylondon.news" rel="preconnect"/>
  <link href="https://i2-prod.mylondon.news" rel="dns-prefetch"/>
  <link href="https://i2-prod.mylondon.news" rel="preconnect"/>
  <link href="https://felix.data.tm-awx.com" rel="dns-prefetch"/>
  <link href="https://felix.data.tm-awx.com" rel="preconnect"/>
  <link href="https://www.googletagmanager.com" rel="dns-prefetch"/>
  <link href="https://www.googletagmanager.com" rel="preconnect"/>
  <link href="https://www.google-analytics.com" rel="dns-prefetch"/>
  <link href="https://www.google-analytics.com" rel="preconnect"/>
  <link href="https://c.go-mpulse.net" rel="dns-prefetch"/>
  <link href="https://c.go-mpulse.net" rel="preconnect"/>
  <link h

In [50]:
table = soup.find('table')
table_rows = table.find_all('tr')
print(table_rows)

[<tr> <td><p>Barking and Dagenham</p></td> <td>437</td> </tr>, <tr> <td>Barnet</td> <td>1143</td> </tr>, <tr> <td>Bexley</td> <td>561</td> </tr>, <tr> <td>Brent</td> <td>1289</td> </tr>, <tr> <td>Bromley</td> <td>959</td> </tr>, <tr> <td>Camden</td> <td>543</td> </tr>, <tr> <td>City of London</td> <td>16</td> </tr>, <tr> <td>Croydon</td> <td>1268</td> </tr>, <tr> <td>Ealing</td> <td>988</td> </tr>, <tr> <td>Enfield</td> <td>763</td> </tr>, <tr> <td>Greenwich</td> <td>592</td> </tr>, <tr> <td>Hackney</td> <td>586</td> </tr>, <tr> <td>Hammersmith and Fulham</td> <td>524</td> </tr>, <tr> <td>Haringey</td> <td>514</td> </tr>, <tr> <td>Harrow</td> <td>841</td> </tr>, <tr> <td>Havering</td> <td>601</td> </tr>, <tr> <td>Hillingdon</td> <td>680</td> </tr>, <tr> <td>Hounslow</td> <td>613</td> </tr>, <tr> <td>Islington</td> <td>389</td> </tr>, <tr> <td>Kensington and Chelsea</td> <td>429</td> </tr>, <tr> <td>Kingston upon Thames</td> <td>411</td> </tr>, <tr> <td>Lambeth</td> <td>1071</td> </tr>,

In [51]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['Borough', 'Confirmed Cases'])


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33 entries, 0 to 32
Data columns (total 2 columns):
Borough            33 non-null object
Confirmed Cases    33 non-null object
dtypes: object(2)
memory usage: 608.0+ bytes


In [53]:
df

,Borough,Confirmed Cases
0,Barking and Dagenham,437
1,Barnet,1143
2,Bexley,561
3,Brent,1289
4,Bromley,959
5,Camden,543
6,City of London,16
7,Croydon,1268
8,Ealing,988
9,Enfield,763


In [54]:
type(df)

pandas.core.frame.DataFrame

### Part 2: Scraping additional information of the different Boroughs in London from a Wikipedia page 
Using Beautiful soup to scrap the latitude and longitiude of the boroughs in London

URL: https://en.wikipedia.org/wiki/List_of_London_boroughs

In [55]:
# getting data from internet
wiki_link ='https://en.wikipedia.org/wiki/List_of_London_boroughs'
wiki_source_page = requests.get(wiki_link).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(wiki_source_page,'xml')
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="UTF-8"/>
  <title>
   List of London boroughs - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XqRVxApAMNIAAdpMGNEAAACH","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_London_boroughs","wgTitle":"List of London boroughs","wgCurRevisionId":953063238,"wgRevisionId":953063238,"wgArticleId":28092685,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from August 2015","Use British English from August 2015","Lists of coordinates"

In [56]:
table_wiki = soup.find('table')
table_rows_wiki = table_wiki.find_all('tr')
print(table_rows_wiki)

[<tr>
<th>Borough
</th>
<th>Inner
</th>
<th>Status
</th>
<th>Local authority
</th>
<th>Political control
</th>
<th>Headquarters
</th>
<th>Area (sq mi)
</th>
<th>Population (2013 est)<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup>
</th>
<th>Co-ordinates
</th>
<th><span style="background:#67BCD3"> Nr. in map </span>
</th></tr>, <tr>
<td><a href="/wiki/London_Borough_of_Barking_and_Dagenham" title="London Borough of Barking and Dagenham">Barking and Dagenham</a> <sup class="reference" id="cite_ref-2"><a href="#cite_note-2">[note 1]</a></sup>
</td>
<td>
</td>
<td>
</td>
<td><a href="/wiki/Barking_and_Dagenham_London_Borough_Council" title="Barking and Dagenham London Borough Council">Barking and Dagenham London Borough Council</a>
</td>
<td><a href="/wiki/Labour_Party_(UK)" title="Labour Party (UK)">Labour</a>
</td>
<td><a href="/wiki/Barking_Town_Hall" title="Barking Town Hall">Town Hall</a>, 1 Town Square
</td>
<td>13.93
</td>
<td>194,352
</td>
<td><span class

In [57]:
data = []
for row in table_rows_wiki:
    data.append([t.text.strip() for t in row.find_all('td')])

df_London = pd.DataFrame(data, columns=['Borough','Inner','Status','Local authority','Political control',
                         'Headquarters','Area (sq mi)','Population (2013 est)[1]','Co-ordinates','Nr. in map'])



In [58]:
df_London

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,None,None,None,None,None,None,None,None,None,None
1,Barking and Dagenham [note 1],,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25
2,Barnet,,,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,31
3,Bexley,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,23
4,Brent,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,12
5,Bromley,,,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,20
6,Camden,Y,,Camden London Borough Council,Labour,"Camden Town Hall, Judd Street",8.40,"229,719",51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W﻿ /...,11
7,Croydon,,,Croydon London Borough Council,Labour,"Bernard Weatherill House, Mint Walk",33.41,"372,752",51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W﻿ /...,19
8,Ealing,,,Ealing London Borough Council,Labour,"Perceval House, 14-16 Uxbridge Road",21.44,"342,494",51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W﻿ /...,13
9,Enfield,,,Enfield London Borough Council,Labour,"Civic Centre, Silver Street",31.74,"320,524",51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W﻿ /...,30


In [59]:
df_London = df_London.replace('note 1','', regex=True) 
df_London = df_London.replace('note 2','', regex=True) 
df_London = df_London.replace('note 3','', regex=True) 
df_London = df_London.replace('note 4','', regex=True) 
df_London = df_London.replace('note 5','', regex=True) 

df_London

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,None,None,None,None,None,None,None,None,None,None
1,Barking and Dagenham [],,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25
2,Barnet,,,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,31
3,Bexley,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,23
4,Brent,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,12
5,Bromley,,,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,20
6,Camden,Y,,Camden London Borough Council,Labour,"Camden Town Hall, Judd Street",8.40,"229,719",51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W﻿ /...,11
7,Croydon,,,Croydon London Borough Council,Labour,"Bernard Weatherill House, Mint Walk",33.41,"372,752",51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W﻿ /...,19
8,Ealing,,,Ealing London Borough Council,Labour,"Perceval House, 14-16 Uxbridge Road",21.44,"342,494",51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W﻿ /...,13
9,Enfield,,,Enfield London Borough Council,Labour,"Civic Centre, Silver Street",31.74,"320,524",51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W﻿ /...,30


In [60]:
type(df_London)

pandas.core.frame.DataFrame

In [61]:
set(df['Borough']) - set(df_London['Borough'])

{'Barking and Dagenham',
 'City of London',
 'Greenwich',
 'Hammersmith and Fulham'}

In [62]:
print("Index of Barking and Dagenham:", df_London.index[df_London['Borough'] == 'Barking and Dagenham []'].tolist())
print("Index of City of London:", df_London.index[df_London['Borough'] == 'City of London []'].tolist())
print("Index of Greenwich:", df_London.index[df_London['Borough'] == 'Greenwich []'].tolist())
print("Index of Hammersmith and Fulham:", df_London.index[df_London['Borough'] == 'Hammersmith and Fulham []'].tolist())

Index of Barking and Dagenham: [1]
Index of City of London: []
Index of Greenwich: [10]
Index of Hammersmith and Fulham: [12]


### Changing the Borough names to match the other data frame

In [63]:
df_London.iloc[1,0] = 'Barking and Dagenham'
df_London.iloc[10,0] = 'Greenwich'
df_London.iloc[12,0] = 'Hammersmith and Fulham'

In [64]:
df_London.drop([0], axis=0, inplace=True)
df_London.reset_index(drop=True, inplace=True)

In [65]:
df_London

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham,,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25
1,Barnet,,,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,31
2,Bexley,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,23
3,Brent,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,12
4,Bromley,,,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,20
5,Camden,Y,,Camden London Borough Council,Labour,"Camden Town Hall, Judd Street",8.40,"229,719",51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W﻿ /...,11
6,Croydon,,,Croydon London Borough Council,Labour,"Bernard Weatherill House, Mint Walk",33.41,"372,752",51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W﻿ /...,19
7,Ealing,,,Ealing London Borough Council,Labour,"Perceval House, 14-16 Uxbridge Road",21.44,"342,494",51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W﻿ /...,13
8,Enfield,,,Enfield London Borough Council,Labour,"Civic Centre, Silver Street",31.74,"320,524",51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W﻿ /...,30
9,Greenwich,Y [],Royal,Greenwich London Borough Council,Labour,"Woolwich Town Hall, Wellington Street",18.28,"264,008",51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E﻿ /...,22


### We can combine both dataframes together

In [44]:
London_Coronavirus_df = pd.merge(df, df_London, on = 'Borough')
London_Coronavirus_df = London_Coronavirus_df.rename(columns = {'Confirmed Cases': 'Confirmed Coronavirus Cases'})

In [45]:
London_Coronavirus_df

,Borough,Confirmed Coronavirus Cases,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham,437,,,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,"194,352",51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E﻿ /...,25
1,Barnet,1143,,,Barnet London Borough Council,Conservative,"North London Business Park, Oakleigh Road South",33.49,"369,088",51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W﻿ /...,31
2,Bexley,561,,,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,"236,687",51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E﻿ /...,23
3,Brent,1289,,,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,"317,264",51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W﻿ /...,12
4,Bromley,959,,,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,"317,899",51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E﻿ /...,20
5,Camden,543,Y,,Camden London Borough Council,Labour,"Camden Town Hall, Judd Street",8.40,"229,719",51°31′44″N 0°07′32″W﻿ / ﻿51.5290°N 0.1255°W﻿ /...,11
6,Croydon,1268,,,Croydon London Borough Council,Labour,"Bernard Weatherill House, Mint Walk",33.41,"372,752",51°22′17″N 0°05′52″W﻿ / ﻿51.3714°N 0.0977°W﻿ /...,19
7,Ealing,988,,,Ealing London Borough Council,Labour,"Perceval House, 14-16 Uxbridge Road",21.44,"342,494",51°30′47″N 0°18′32″W﻿ / ﻿51.5130°N 0.3089°W﻿ /...,13
8,Enfield,763,,,Enfield London Borough Council,Labour,"Civic Centre, Silver Street",31.74,"320,524",51°39′14″N 0°04′48″W﻿ / ﻿51.6538°N 0.0799°W﻿ /...,30
9,Greenwich,592,Y [],Royal,Greenwich London Borough Council,Labour,"Woolwich Town Hall, Wellington Street",18.28,"264,008",51°29′21″N 0°03′53″E﻿ / ﻿51.4892°N 0.0648°E﻿ /...,22
